# Standardizing Event Data for DeRisk

DeRisk currently works with event data in a raw format where relevant data (like the user, token, amount, etc.) is saved in a dictionary or list-like structure in one of the columns. To better extract information from the database, it is helpful to have a unified data structure where all relevant pieces of information are saved in separate columns. This allows for easy querying of all events of a given type (deposit, withdrawal, liquidation) for a given user, lending protocol, et

The following steps outline the process to take a sample of events and convert them to a standardized format that can be used to store information about any type of event and any ing protocolv')


In [1]:
#import libraries
import pandas as pd 

#display settings
pd.set_option('display.max_colwidth', None)

1.  Load event data from Paraquet file

In [2]:
def load_protocol_data(protocols: list[str]) -> pd.DataFrame:
    """
    Load data from Google Storage for the specified protocols and combine them into a single DataFrame.
    
    Parameters:
    protocols (list[str]): A list of protocol names to load data for.
    
    Returns:
    pd.DataFrame: A DataFrame containing the combined data from all specified protocols.
    
    """    
    combined_protocols_df = pd.DataFrame()
    
    for protocol in protocols:
        # Read from google storage
        url = f"https://storage.googleapis.com/derisk-persistent-state/{protocol}_data/events_sample.parquet"      
        print(f"Processing {protocol} from Google Storage...")
        df_protocol = pd.read_parquet(url)
        
        # Read from SQL DB
        ## Uncomment the following code block and comment the above three lines to load data from a local database
        # connection = src.db.establish_connection()
        # query = f"SELECT * FROM {protocol}_data" # Ensure table name is correct
        # print(f"Processing {protocol} from local database...")
        # df_protocol = pd.read_sql(query, con = connection)
        # connection.close()
        
        df_protocol['Protocol'] = protocol
        combined_protocols_df = pd.concat([combined_protocols_df, df_protocol], ignore_index=True)
    return combined_protocols_df

# List of protocols
PROTOCOLS = ['zklend', 'nostra_alpha', 'nostra_mainnet', 'hashstack_v0', 'hashstack_v1']

# Load the data
df_events = load_protocol_data(PROTOCOLS)
print(f"Combined dataframe shape: {df_events.shape}")

Processing zklend from Google Storage...
Processing nostra_alpha from Google Storage...
Processing nostra_mainnet from Google Storage...
Processing hashstack_v0 from Google Storage...
Processing hashstack_v1 from Google Storage...
Combined dataframe shape: (305033, 11)


In [3]:
df_events.head()

,block_hash,block_number,transaction_hash,event_index,from_address,keys,data,timestamp,key_name,Protocol,order
0,0x01ff1d0fe076a0fec3ce0b69bd1c3b6b9f1dfb2851277114b6e946695ca9684c,48689,0x01cb9af2e303d758d96c13fa365c4a7aa66e9ba94150311faa5da9b7afef4035,12,0x04c0a5193d58f74fbace4b74dcf65481e734ed1714121bdc571da345540efa05,"b'[""0x30c296ae369716818de77cb5b71ce9cda7cc2c0e8456f474e0abb1ae8d017da""]'","b'[""0x3fe2b97c1fd336e750087d68b9b867997fd64a2661ff3ca5a7c771641e8e7ac"",""0x33b2e3c9fd0803ce8000000"",""0x33b2e3c9fd0803ce8000000""]'",1683226849,AccumulatorsSync,zklend,NaN
1,0x01ff1d0fe076a0fec3ce0b69bd1c3b6b9f1dfb2851277114b6e946695ca9684c,48689,0x01cb9af2e303d758d96c13fa365c4a7aa66e9ba94150311faa5da9b7afef4035,15,0x04c0a5193d58f74fbace4b74dcf65481e734ed1714121bdc571da345540efa05,"b'[""0x30c296ae369716818de77cb5b71ce9cda7cc2c0e8456f474e0abb1ae8d017da""]'","b'[""0x68f5c6a61780768455de69077e07e89787839bf8166decfbf92b645209c0fb8"",""0x33b2e3c9fd0803ce8000000"",""0x33b2e3c9fd0803ce8000000""]'",1683226849,AccumulatorsSync,zklend,NaN
2,0x01ff1d0fe076a0fec3ce0b69bd1c3b6b9f1dfb2851277114b6e946695ca9684c,48689,0x01cb9af2e303d758d96c13fa365c4a7aa66e9ba94150311faa5da9b7afef4035,18,0x04c0a5193d58f74fbace4b74dcf65481e734ed1714121bdc571da345540efa05,"b'[""0x30c296ae369716818de77cb5b71ce9cda7cc2c0e8456f474e0abb1ae8d017da""]'","b'[""0xda114221cb83fa859dbdb4c44beeaa0bb37c7537ad5ae66fe5e0efd20e6eb3"",""0x33b2e3c9fd0803ce8000000"",""0x33b2e3c9fd0803ce8000000""]'",1683226849,AccumulatorsSync,zklend,NaN
3,0x01ff1d0fe076a0fec3ce0b69bd1c3b6b9f1dfb2851277114b6e946695ca9684c,48689,0x01cb9af2e303d758d96c13fa365c4a7aa66e9ba94150311faa5da9b7afef4035,6,0x04c0a5193d58f74fbace4b74dcf65481e734ed1714121bdc571da345540efa05,"b'[""0x30c296ae369716818de77cb5b71ce9cda7cc2c0e8456f474e0abb1ae8d017da""]'","b'[""0x49d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7"",""0x33b2e3c9fd0803ce8000000"",""0x33b2e3c9fd0803ce8000000""]'",1683226849,AccumulatorsSync,zklend,NaN
4,0x01ff1d0fe076a0fec3ce0b69bd1c3b6b9f1dfb2851277114b6e946695ca9684c,48689,0x01cb9af2e303d758d96c13fa365c4a7aa66e9ba94150311faa5da9b7afef4035,9,0x04c0a5193d58f74fbace4b74dcf65481e734ed1714121bdc571da345540efa05,"b'[""0x30c296ae369716818de77cb5b71ce9cda7cc2c0e8456f474e0abb1ae8d017da""]'","b'[""0x53c91253bc9682c04929ca02ed00b3e423f6710d2ee7e0d5ebb06f3ecf368a8"",""0x33b2e3c9fd0803ce8000000"",""0x33b2e3c9fd0803ce8000000""]'",1683226849,AccumulatorsSync,zklend,NaN


2. Define a Function to Transform Each Row
Define a function that decodes the keys and data columns, converts the hexadecimal strings to integers, and structures the information into a standardized format.

### For zklend Protocol, these are the events and their relevant data.
- for accumulators_sync event: token is in data[0], lending_accumulator is in data[1], debt_accumulator is in data[2]
- for deposit event: user is in data[0], token is in data[1], face_amount is in data[2]
- for collateral_enabled event: user is in data[0], token is in data[1]
- for collateral_disabled event: user is in data[0], token is in data[1]
- for withdrawal event: user is in data[0], token is in data[1], face_amount is in data[2]
- for borrowing event: user is in data[0], token is in data[1], raw_amount is in data[2], face_amount is in data[3]
- for repayment event: user is in data[1], token is in data[2], raw_amount is in data[3]
- for liquidation event: user is in data[1], debt_token is in data[2], debt_raw_amnt is in data[3], collateral_token is in data[5], collateral_face_amnt is in data[6]

In [4]:
# Define a function to decode bytes, convert to list, and extract specific elements based on the event
def extract_zklend(row):
    
    def hex_to_int(hex_str):
        return int(hex_str, 16)
    
    try:
        # Decode the bytes to a string and convert the string to an actual list
        data_str = row['data'].decode('utf-8')
        data_list = eval(data_str)
        
        if row['key_name'] == 'Deposit':
            return pd.Series({
                'user': data_list[0],
                'token': data_list[1],
                'face_amount': hex_to_int(data_list[2])
            })
        elif row['key_name'] == 'Withdrawal':
            return pd.Series({
                'user': data_list[0],
                'token': data_list[1],
                'face_amount': hex_to_int(data_list[2])
            })
        elif row['key_name'] == 'Liquidation':
            return pd.Series({
                'user': data_list[1],
                'debt_token': data_list[2],
                'debt_raw_amount': hex_to_int(data_list[3]),
                'collateral_token': data_list[5],
                'collateral_face_amount': data_list[6]
            })
        elif row['key_name'] == 'CollateralEnabled':
            return pd.Series({
                'user': data_list[0],
                'token': data_list[1]
            })
        elif row['key_name'] == 'CollateralDisabled':
            return pd.Series({
                'user': data_list[0],
                'token': data_list[1]
            })
        elif row['key_name'] == 'Borrowing':
            return pd.Series({
                'user': data_list[0],
                'token': data_list[1],
                'raw_amount': hex_to_int(data_list[2]),
                'face_amount': hex_to_int(data_list[3])
            })
        elif row['key_name'] == 'Repayment':
            return pd.Series({
                'user': data_list[1],
                'token': data_list[2],
                'raw_amount': hex_to_int(data_list[3])
            })
        elif row['key_name'] == 'AccumulatorsSync':
            return pd.Series({
                'token': data_list[0],
                'lending_accumulator': data_list[1],
                'debt_accumulator': data_list[2]
            })
        else:
            return pd.Series()
    except Exception as e:
        # Handle any parsing errors
        return pd.Series()

# Filter the dataset for 'zklend' protocol
zk_data = df_events[df_events['Protocol'] == 'zklend']

# Apply the extraction function to each row
extracted_elements = zk_data.apply(extract_zklend, axis=1)

# Concatenate the extracted elements with the original dataframe
standard_zk = pd.concat([zk_data[['block_hash', 'block_number', 'transaction_hash', 'timestamp', 'key_name']], extracted_elements], axis=1)

# display the dataframe
# standard_zk.head()

# Uncomment the following below to Save the resulting dataframe to a new CSV file
# output_file_path = 'extracted_standard_zk_events.csv'
# standard_zk.to_csv(output_file_path, index=False)

### For nostra_alpha protocol, the events are mapped on tokens, here are the events.

non_interest_bearing_collateral_mint_event: deposit
non_interest_bearing_collateral_burn_event: 
interest_bearing_collateral_mint_event: deposit
interest_bearing_collateral_burn_event
debt_mint_event: borrowing
debt_burn_event : repayment


In [5]:
# Define the mapping for token addresses and their types
TOKEN_MAPPING = {
    '0x0553cea5d1dc0e0157ffcd36a51a0ced717efdadd5ef1b4644352bb45bd35453': 'non_interest_bearing_collateral',
    '0x047e794d7c49c49fd2104a724cfa69a92c5a4b50a5753163802617394e973833': 'non_interest_bearing_collateral',
    '0x003cd2066f3c8b4677741b39db13acebba843bbbaa73d657412102ab4fd98601': 'non_interest_bearing_collateral',
    '0x04403e420521e7a4ca0dc5192af81ca0bb36de343564a9495e11c8d9ba6e9d17': 'non_interest_bearing_collateral',
    '0x06b59e2a746e141f90ec8b6e88e695265567ab3bdcf27059b4a15c89b0b7bd53': 'non_interest_bearing_collateral',
    '0x070f8a4fcd75190661ca09a7300b7c93fab93971b67ea712c664d7948a8a54c6': 'interest_bearing_collateral',
    '0x029959a546dda754dc823a7b8aa65862c5825faeaaf7938741d8ca6bfdc69e4e': 'interest_bearing_collateral',
    '0x055ba2baf189b98c59f6951a584a3a7d7d6ff2c4ef88639794e739557e1876f0': 'interest_bearing_collateral',
    '0x01ac55cabf2b79cf39b17ba0b43540a64205781c4b7850e881014aea6f89be58': 'interest_bearing_collateral',
    '0x00687b5d9e591844169bc6ad7d7256c4867a10cee6599625b9d78ea17a7caef9': 'interest_bearing_collateral',
    '0x040b091cb020d91f4a4b34396946b4d4e2a450dbd9410432ebdbfe10e55ee5e5': 'debt',
    '0x03b6058a9f6029b519bc72b2cc31bcb93ca704d0ab79fec2ae5d43f79ac07f7a': 'debt',
    '0x065c6c7119b738247583286021ea05acc6417aa86d391dcdda21843c1fc6e9c6': 'debt',
    '0x0362b4455f5f4cc108a5a1ab1fd2cc6c4f0c70597abb541a99cf2734435ec9cb': 'debt',
    '0x075b0d87aca8dee25df35cdc39a82b406168fa23a76fc3f03abbfdc6620bb6d7': 'debt'
}

# Define the event type mapping based on the token type and key name
EVENT_MAPPING = {
    'non_interest_bearing_collateral': {
        'Mint': 'deposit',
        'Burn': 'withdrawal'
    },
    'interest_bearing_collateral': {
        'Mint': 'deposit',
        'Burn': 'withdrawal'
    },
    'debt': {
        'Mint': 'borrowing',
        'Burn': 'repayment'
    }
}

def get_event_type(token_address, key_name):
    token_type = TOKEN_MAPPING.get(token_address.lower())
    if token_type:
        return EVENT_MAPPING.get(token_type, {}).get(key_name)
    return 'interest rate updated'  # Default to 'interest rate updated' if not matched

def extract_nostra_alpha(row):
    def hex_to_int(hex_str):
        return int(hex_str, 16)
    
    try:
        # Decode the bytes to a string and convert the string to an actual list
        data_str = row['data'].decode('utf-8')
        data_list = eval(data_str)
        event_type = get_event_type(row['from_address'], row['key_name'])
        
        if row['key_name'] in ['Mint', 'Burn']:
            return pd.Series({
                'user': data_list[0],
                'token': row['from_address'],
                'face_amount': hex_to_int(data_list[1]),
                'event_type': event_type
            })
        elif row['key_name'] == 'InterestStateUpdated':
            return pd.Series({
                'debt_token': data_list[0],
                'lending_rate': data_list[1],
                'borrow_rate': data_list[3],
                'lend_index': data_list[5],
                'borrow_index': data_list[7],
                
            })
        else:
            return pd.Series()  # Return an empty series for unmatched events
    except Exception as e:
        # Handle any parsing errors
        return pd.Series()  # Return an empty series in case of error

# Filter the dataset for 'nostra_alpha' protocol
nostra_alpha_data = df_events[df_events['Protocol'] == 'nostra_alpha']

# Apply the extraction function to each row
extracted_elements = nostra_alpha_data.apply(extract_nostra_alpha, axis=1)

# Concatenate the extracted elements with the original dataframe
standard_nalpha = pd.concat([nostra_alpha_data[['block_hash', 'block_number', 'transaction_hash', 'timestamp']], extracted_elements], axis=1)

# Uncomment the following to save the resulting dataframe to a new CSV file
# output_file_path = 'extracted_standard_nalpha_events.csv'
# standard_nalpha.to_csv(output_file_path, index=False)

# Show a sample of the resulting dataframe
# print(standard_nalpha.head())

### For hashstack_v0 protocol these are the events and their relevant data:
- for new loan: loan_id is in data[0]int, user is in data[1], debt_token is in data[2], commitment is in data[3], debt_face_amount is in data[4]int, borrowed_collateral_token is in data[6],borrowed_collateral_face_amount is in data[7]int, is_loan_withdrawn is in data[9] debt_category is in data[10]int, state is in data[11], l3_integration is in data[12], created_at is in data[13], [`collateral`] `market` is in data[14], amount is in data[15], current_amount is in data[17], commitment is repeated in data_list[19]( but the values do not all match), timelock_validity is in data_list[20], is_timelock_activated is in data_list[21], activation_time is in data_list[22], timestamp is in data_list[23]
                
- for loan_withdrawal: id (loan_record): Located in data[0], owner: Located in data[1], market: Located in data[2], commitment: Located in data[3], amount: Located in data[4], current_market: Located in data[6], current_amount: Located in data[7], is_loan_withdrawn: Located in data[9], debt_category: Located in data[10], state: Located in data[11], l3_integration: Located in data[12], created_at: Located in data[13], amount_withdrawn: Located in data[14] (inside amount_withdrawn), timestamp: Located in data[16] (inside timestamp).

- for loan_interest_deducted: market (collateral_record): Located in data[0], amount: Located in data[1], (empty): Located in data[2] (not used), current_amount: Located in data[3], (empty): Located in data[4] (not used), commitment: Located in data[5], timelock_validity: Located in data[6], is_timelock_activated: Located in data[7], activation_time: Located in data[8], accrued_interest (inside accrued_interest): Located in data[9], (empty): Located in data[10] (not used), loan_id (inside loan_id): Located in data[11], amount_withdrawn (inside amount_withdrawn): Located in data[12], (empty): Located in data[13] (not used), timestamp (inside timestamp): Located in data[14]
  
- for loan_swap: old_loan_id : Located in data[0], old_loan_owner: Located in data[1], old_loan_market: Located in data[2], old_loan_commitment: Located in data[3], old_loan_amount: Located in data[4], (empty): Located in data[5] (not used), old_loan_current_market: Located in data[6], old_loan_current_amount: Located in data[7], (empty): Located in data[8] (not used), old_loan_is_loan_withdrawn: Located in data[9], old_loan_debt_category: Located in data[10], old_loan_state: Located in data[11], old_loan_l3_integration: Located in data[12], old_loan_created_at: Located in data[13], new_loan_id: Located in data[14], new_loan_owner: Located in data[15], new_loan_market: Located in data[16], new_loan_commitment: Located in data[17], new_loan_amount: Located in data[18], (empty): Located in data[19] (not used), new_loan_current_market: Located in data[20], new_loan_current_amount: Located in data[21], (empty): Located in data[22] (not used), new_loan_is_loan_withdrawn: Located in data[23], new_loan_debt_category: Located in data[24], new_loan_state: Located in data[25], new_loan_l3_integration: Located in data[26], new_loan_created_at: Located in data[27], timestamp: Located in data[28].

  
- for collateral_added: collateral_market: Located in data[0], collateral_amount: Located in data[1], (empty): Located in data[2] (not used), collateral_current_amount: Located in data[3], (empty): Located in data[4] (not used), collateral_commitment: Located in data[5], collateral_timelock_validity: Located in data[6], collateral_is_timelock_activated: Located in data[7], collateral_activation_time: Located in data[8], collateral_loan_id: Located in data[9], collateral_amount_added: Located in data[10], (empty): Located in data[11] (not used), collateral_timestamp: Located in data[12]
  
- for loan_repaid: loan_id: Located in data[0], loan_owner: Located in data[1], loan_market: Located in data[2], loan_commitment: Located in data[3], loan_amount: Located in data[4], (empty): Located in data[5] (not used), loan_current_market: Located in data[6], loan_current_amount: Located in data[7], (empty): Located in data[8] (not used), loan_is_loan_withdrawn: Located in data[9], loan_debt_category: Located in data[10], loan_state: Located in data[11], loan_l3_integration: Located in data[12], loan_created_at: Located in data[13], loan_timestamp: Located in data[14].

- for collateral_withdrawal: collateral_market: Located in data[0], collateral_amount: Located in data[1], (empty): Located in data[2] (not used), collateral_current_amount: Located in data[3], (empty): Located in data[4] (not used), collateral_commitment: Located in data[5], collateral_timelock_validity: Located in data[6], collateral_is_timelock_activated: Located in data[7], collateral_activation_time: Located in data[8], collateral_loan_id: Located in data[9], collateral_amount_withdrawn: Located in data[10], (empty): Located in data[11] (not used), collateral_timestamp: Located in data[12].

- for liquidated_event: loan_id is in data[0]int, user is in data[1], debt_token is in data[2], commitment is in data[3],amount is in data[4]int,  borrowed_collateral_token is in data[6], borrowed_collateral_face_amount is in data[7]int, loan_withdrawn is in data[9], debt_category is in data[10]int, state is in data[11], l3_integration is in data[12], created_at is in data[13], liquidator is in data[14], timestamp is in data[15]
  

In [6]:
def extract_hashstack_v0(row):
    
    def hex_to_int(hex_str):
        return int(hex_str, 16)
    
    try:
        # Decode the bytes to a string and convert the string to an actual list
        data_str = row['data'].decode('utf-8')
        data_list = eval(data_str)
        
        if row['key_name'] == 'new_loan':
            return pd.Series({
                'loan_id': hex_to_int(data_list[0]),
                'user': data_list[1],
                'debt_token': data_list[2],
                'commitment': data_list[3],
                'debt_face_amount': hex_to_int(data_list[4]),
                'borrowed_collateral_token': data_list[6],
                'borrowed_collateral_face_amount': hex_to_int(data_list[7]),
                'is_loan_withdrawn': data_list[9],
                'debt_category': hex_to_int(data_list[10]),
                'state': data_list[11],
                'l3_integration': data_list[12],
                'created_at': data_list[13],
                'collateral_market': data_list[14],
                'amount': data_list[15],
                'current_amount': data_list[17],
                'timelock_validity': data_list[20],
                'is_timelock_activated': data_list[21],
                'activation_time': data_list[22],
                'timestamp': data_list[23]
            })
        elif row['key_name'] == 'loan_withdrawal':
            return pd.Series({
                'id': data_list[0],
                'owner': data_list[1],
                'market': data_list[2],
                'commitment': data_list[3],
                'amount': data_list[4],
                'current_market': data_list[6],
                'current_amount': data_list[7],
                'is_loan_withdrawn': data_list[9],
                'debt_category': data_list[10],
                'state': data_list[11],
                'l3_integration': data_list[12],
                'created_at': data_list[13],
                'amount_withdrawn': data_list[14],
                'timestamp': data_list[16]
            })
        elif row['key_name'] == 'loan_interest_deducted':
            return pd.Series({
                'market': data_list[0],
                'amount': data_list[1],
                'current_amount': data_list[3],
                'commitment': data_list[5],
                'timelock_validity': data_list[6],
                'is_timelock_activated': data_list[7],
                'activation_time': data_list[8],
                'accrued_interest': data_list[9],
                'loan_id': data_list[11],
                'amount_withdrawn': data_list[12],
                'timestamp': data_list[14]
            })
        elif row['key_name'] == 'loan_swap':
            return pd.Series({
                'old_loan_id': data_list[0],
                'old_loan_owner': data_list[1],
                'old_loan_market': data_list[2],
                'old_loan_commitment': data_list[3],
                'old_loan_amount': data_list[4],
                'old_loan_current_market': data_list[6],
                'old_loan_current_amount': data_list[7],
                'old_loan_is_loan_withdrawn': data_list[9],
                'old_loan_debt_category': data_list[10],
                'old_loan_state': data_list[11],
                'old_loan_l3_integration': data_list[12],
                'old_loan_created_at': data_list[13],
                'new_loan_id': data_list[14],
                'new_loan_owner': data_list[15],
                'new_loan_market': data_list[16],
                'new_loan_commitment': data_list[17],
                'new_loan_amount': data_list[18],
                'new_loan_current_market': data_list[20],
                'new_loan_current_amount': data_list[21],
                'new_loan_is_loan_withdrawn': data_list[23],
                'new_loan_debt_category': data_list[24],
                'new_loan_state': data_list[25],
                'new_loan_l3_integration': data_list[26],
                'new_loan_created_at': data_list[27],
                'timestamp': data_list[28]
            })
        elif row['key_name'] == 'collateral_added':
            return pd.Series({
                'collateral_market': data_list[0],
                'collateral_amount': data_list[1],
                'collateral_current_amount': data_list[3],
                'collateral_commitment': data_list[5],
                'collateral_timelock_validity': data_list[6],
                'collateral_is_timelock_activated': data_list[7],
                'collateral_activation_time': data_list[8],
                'collateral_loan_id': data_list[9],
                'collateral_amount_added': data_list[10],
                'collateral_timestamp': data_list[12]
            })
        elif row['key_name'] == 'loan_repaid':
            return pd.Series({
                'loan_id': data_list[0],
                'loan_owner': data_list[1],
                'loan_market': data_list[2],
                'loan_commitment': data_list[3],
                'loan_amount': data_list[4],
                'loan_current_market': data_list[6],
                'loan_current_amount': data_list[7],
                'loan_is_loan_withdrawn': data_list[9],
                'loan_debt_category': data_list[10],
                'loan_state': data_list[11],
                'loan_l3_integration': data_list[12],
                'loan_created_at': data_list[13],
                'loan_timestamp': data_list[14]
            })
        elif row['key_name'] == 'collateral_withdrawal':
            return pd.Series({
                'collateral_market': data_list[0],
                'collateral_amount': data_list[1],
                'collateral_current_amount': data_list[3],
                'collateral_commitment': data_list[5],
                'collateral_timelock_validity': data_list[6],
                'collateral_is_timelock_activated': data_list[7],
                'collateral_activation_time': data_list[8],
                'collateral_loan_id': data_list[9],
                'collateral_amount_withdrawn': data_list[10],
                'collateral_timestamp': data_list[12]
            })
        elif row['key_name'] == 'liquidated_event':
            return pd.Series({
                'loan_id': hex_to_int(data_list[0]),
                'user': data_list[1],
                'debt_token': data_list[2],
                'commitment': data_list[3],
                'amount': hex_to_int(data_list[4]),
                'borrowed_collateral_token': data_list[6],
                'borrowed_collateral_face_amount': hex_to_int(data_list[7]),
                'loan_withdrawn': data_list[9],
                'debt_category': hex_to_int(data_list[10]),
                'state': data_list[11],
                'l3_integration': data_list[12],
                'created_at': data_list[13],
                'liquidator': data_list[14],
                'timestamp': data_list[15]
            })
        else:
            return pd.Series()
    except Exception as e:
        # Handle any parsing errors
        return pd.Series()

# Filter the dataset for 'hashstack_v0' protocol
hashstack_v0_data = df_events[df_events['Protocol'] == 'hashstack_v0']

# Apply the extraction function to each row
extracted_elements = hashstack_v0_data.apply(extract_hashstack_v0, axis=1)

# Concatenate the extracted elements with the original dataframe
standard_hashstack_v0 = pd.concat([hashstack_v0_data[['block_hash', 'block_number', 'transaction_hash', 'timestamp', 'key_name']], extracted_elements], axis=1)

# Display the resulting dataframe
# standard_hashstack_v0.head()

# Uncomment the following below to Save the resulting dataframe to a new CSV file
# output_file_path = 'extracted_hashstack_v0_events.csv'
# standard_hashstack_v0.to_csv(output_file_path, index=False)

### for hashstack_v1 protocol, these are the following events and their relevant data:
- for new_loan event: loan_id is in data[0]int, borrower is in data[1], market is in data[2], loan_amount is in data[3]int, current_market is in data[5], current_amount is in data[6], state is in data[8], l3_integration is in data[9], l3_category is in data[10], loan_created_at is in data[11], collateral_loan_id is in data[12], collateral_token is in data[13], collateral_amount is in data[14]int, collateral_created_at is in data[16], timestamp is in data[17]
  
- for loan_spent: old_loan_id is in data[0]int, borrower is in data[1], market is in data[2], old_loan_amount is in data[3]int, current_market is in data[5], current_amount is in data[6], state is in data[8], l3_integration is in data[9], l3_category is in data[10], loan_created_at is in data[11], new_loan_id is in data[12], new_borrower is in data[13], new_market is in data[14], new_loan_amount is in data[15]int, new_current_market is in data[17], new_current_amount is in data[18], new_state is in data[20], new_l3_integration is in data[21], new_l3_category is in data[22], new_loan_created_at is in data[23], timestamp is in data[24]

- for collateral_added: loan_id is in data[0]int, collateral_token is in data[1], amount is in data[2], created_at is in data[4], amount_added is in data[5], timestamp is in data[6]

- for loan_repaid: loan_id is in data[0], borrower is in data[1], market is located in data[2], amount is located in data[3], current_market is located in data[5], current_amount is located in data[6], state is located in data[8], l3_integration is located in data[9], l3_category is located in data[10], created_at is located in data[11], new_ loan is located in data[12], new_borrower is located in data[13], new_market is in data[14], new_amount is in data[15], new_current_mark is in data[17], new_current_amount[18], new_state is in data[20], new_l3_integration is in data[21], new_l3_category is in data[22], new_created_at is in data[22], collateral_loan_id is in data[23], collateral_token is in data[24], collateral_amount is in data[25], collateral_created_at is in data[27], totalUserDebt is in data[28], timestamp is in data[29]
- for loan_transfered: loan_id is in data[0]int, sender is in data[1], reciever is in data[2], timestamp is in data[3]

In [7]:
def extract_hashstack_v1(row):
    
    def hex_to_int(hex_str):
        return int(hex_str, 16)
    
    try:
        # Decode the bytes to a string and convert the string to an actual list
        data_str = row['data'].decode('utf-8')
        data_list = eval(data_str)
        
        if row['key_name'] == 'new_loan':
            return pd.Series({
                'loan_id': hex_to_int(data_list[0]),
                'borrower': data_list[1],
                'market': data_list[2],
                'loan_amount': hex_to_int(data_list[3]),
                'current_market': data_list[5],
                'current_amount': data_list[6],
                'state': data_list[8],
                'l3_integration': data_list[9],
                'l3_category': data_list[10],
                'loan_created_at': data_list[11],
                'collateral_loan_id': hex_to_int(data_list[12]),
                'collateral_token': data_list[13],
                'collateral_amount': hex_to_int(data_list[14]),
                'collateral_created_at': data_list[16],
                'timestamp': data_list[17]
            })
        elif row['key_name'] == 'loan_spent':
            return pd.Series({
                'old_loan_id': hex_to_int(data_list[0]),
                'borrower': data_list[1],
                'market': data_list[2],
                'old_loan_amount': hex_to_int(data_list[3]),
                'current_market': data_list[5],
                'current_amount': data_list[6],
                'state': data_list[8],
                'l3_integration': data_list[9],
                'l3_category': data_list[10],
                'loan_created_at': data_list[11],
                'new_loan_id': data_list[12],
                'new_borrower': data_list[13],
                'new_market': data_list[14],
                'new_loan_amount': hex_to_int(data_list[15]),
                'new_current_market': data_list[17],
                'new_current_amount': data_list[18],
                'new_state': data_list[20],
                'new_l3_integration': data_list[21],
                'new_l3_category': data_list[22],
                'new_loan_created_at': data_list[23],
                'timestamp': data_list[24]
            })
        elif row['key_name'] == 'collateral_added':
            return pd.Series({
                'loan_id': hex_to_int(data_list[0]),
                'collateral_token': data_list[1],
                'amount': data_list[2],
                'created_at': data_list[4],
                'amount_added': data_list[5],
                'timestamp': data_list[6]
            })
        elif row['key_name'] == 'loan_repaid':
            return pd.Series({
                'loan_id': data_list[0],
                'borrower': data_list[1],
                'market': data_list[2],
                'amount': data_list[3],
                'current_market': data_list[5],
                'current_amount': data_list[6],
                'state': data_list[8],
                'l3_integration': data_list[9],
                'l3_category': data_list[10],
                'created_at': data_list[11],
                'new_loan_id': data_list[12],
                'new_borrower': data_list[13],
                'new_market': data_list[14],
                'new_amount': data_list[15],
                'new_current_market': data_list[17],
                'new_current_amount': data_list[18],
                'new_state': data_list[20],
                'new_l3_integration': data_list[21],
                'new_l3_category': data_list[22],
                'new_created_at': data_list[23],
                'collateral_loan_id': data_list[24],
                'collateral_token': data_list[25],
                'collateral_amount': data_list[26],
                'collateral_created_at': data_list[27],
                'totalUserDebt': data_list[28],
                'timestamp': data_list[29]
            })
        elif row['key_name'] == 'loan_transfered':
            return pd.Series({
                'loan_id': hex_to_int(data_list[0]),
                'sender': data_list[1],
                'receiver': data_list[2],
                'timestamp': data_list[3]
            })
        else:
            return pd.Series()
    except Exception as e:
        # Handle any parsing errors
        return pd.Series()

# Filter the dataset for 'hashstack_v1' protocol
hashstack_v1_data = df_events[df_events['Protocol'] == 'hashstack_v1']

# Apply the extraction function to each row
extracted_elements = hashstack_v1_data.apply(extract_hashstack_v1, axis=1)

# Concatenate the extracted elements with the original dataframe
standard_hashstack_v1 = pd.concat([hashstack_v1_data[['block_hash', 'block_number', 'transaction_hash', 'timestamp', 'key_name']], extracted_elements], axis=1)

# Display the resulting dataframe
standard_hashstack_v1.head()

# Uncomment the following below to Save the resulting dataframe to a new CSV file
# output_file_path = 'extracted_hashstack_v1_events.csv'
# standard_hashstack_v1.to_csv(output_file_path, index=False)

,block_hash,block_number,transaction_hash,timestamp,key_name,amount,amount_added,borrower,collateral_amount,collateral_created_at,...,new_loan_amount,new_loan_created_at,new_loan_id,new_market,new_state,old_loan_amount,old_loan_id,state,timestamp,totalUserDebt
300780,0x068b04151068a7acc1f72c9b2023e176037cb6bce171a5e3abaf6fb961f4e1a2,273262,0x0676e3ae850f6fae3da1269b9c390f74c26eb1948ca2bd1e16a47fa9399ca84d,1695977183,new_loan,NaN,NaN,0x251e01d33f75076732f5c7c671d1e6918f3cd0f36a4dedcc35e642297f30d2a,2000000,0x65168edf,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0x1,0x65168edf,NaN
300781,0x06c263286120af7afb020f535cd53e17da319df8435058d76450bdcd3ebaecb3,273300,0x06ebb5723abf3878214b66bd4bbe7087280707da7231b074a1bcfba8b5ec78d7,1695977763,new_loan,NaN,NaN,0x251e01d33f75076732f5c7c671d1e6918f3cd0f36a4dedcc35e642297f30d2a,2000000,0x65169123,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0x1,0x65169123,NaN
300782,0x07d0f1faf834b7906e4de54c440d0aa35112bc1a243c12e63e4f8b6006e857df,273372,0x0051f75ef1e08f70d1c8efe7866384d026aa0ca092ded8bd1c903aac0478b990,1695978978,loan_spent,NaN,NaN,0x251e01d33f75076732f5c7c671d1e6918f3cd0f36a4dedcc35e642297f30d2a,NaN,NaN,...,4500000,0x65168edf,0x1,0x12b8185e237dd0340340faeb3351dbe53f8a42f5a9bf974ddf90ced56e301c7,0x2,4500000,1.0,0x1,0x651695e2,NaN
300783,0x06f650deb66bb602f02bd87f29aa7c28909b126f9d7c738f8a3ad200baa888c4,273387,0x01bf0af4c1922e2b8cc523e6e088d72885701e9416058a3e6bcc2bcf7f0bd5d0,1695979201,collateral_added,0x3d08fe,0x1e847e,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0x0,NaN
300784,0x06384cf4b11477d048623b43e45970c5a4a10653706547b76e92096f7d421774,273396,0x0389888c9ef7dfe1491a3d15afa79b565bc52dccb01d512693f786f6c61fdc0f,1695979342,collateral_added,0x4c4b3e,0xf4240,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0x0,NaN


### for nostra_mainnet, the same extraction will be applied as with nostra_alpha.

In [8]:
# Filter the dataset for 'nostra_alpha' protocol
nostra_mainnet_data = df_events[df_events['Protocol'] == 'nostra_mainnet']

# Apply the extraction function to each row
extracted_elements = nostra_mainnet_data.apply(extract_nostra_alpha, axis=1)

# Concatenate the extracted elements with the original dataframe
standard_mainnet = pd.concat([nostra_mainnet_data[['block_hash', 'block_number', 'transaction_hash', 'timestamp', 'key_name']], extracted_elements], axis=1)

# Display the resulting dataframe
standard_mainnet.head()

# Uncomment the following below to Save the resulting dataframe to a new CSV file
# output_file_path = 'extracted_standard_mainnet_events.csv'
# standard_mainnet.to_csv(output_file_path, index=False)

,block_hash,block_number,transaction_hash,timestamp,key_name,borrow_index,borrow_rate,debt_token,event_type,face_amount,lend_index,lending_rate,token,user
200000,0x051eddd5b8502044947dc35d687caa2b2634d3dbb670d3f1f28654fffdcdc892,169098,0x00489af46e28392d1c3e4007476328ba4ccf4bd84f4f5565fda0888d5518a70b,1692858419,Mint,NaN,NaN,NaN,interest rate updated,1000000000000000000,NaN,NaN,0x04f18ffc850cdfa223a530d7246d3c6fc12a5969e0aa5d4a88f470f5fe6c46e9,0x5295882956890bb9c05e54c1e7efa903a4a21083ebd6a3c175507f74e549574
200001,0x051eddd5b8502044947dc35d687caa2b2634d3dbb670d3f1f28654fffdcdc892,169098,0x00489af46e28392d1c3e4007476328ba4ccf4bd84f4f5565fda0888d5518a70b,1692858419,InterestStateUpdated,0xde0b6b3a7640000,0x0,0x66037c083c33330a8460a65e4748ceec275bbf5f28aa71b686cbc0010e12597,NaN,NaN,0xde0b6b3a7640000,0x0,NaN,NaN
200002,0x0068928198ae9c350e91761752850081262aede8cabd5824341c905b5750fae5,169911,0x01bf3b09632626f8cc8ed46a1b394df8ddf2776b5f04794ad740e335b1ec5b9a,1692881265,Mint,NaN,NaN,NaN,interest rate updated,100000,NaN,NaN,0x05dcd26c25d9d8fd9fc860038dcb6e4d835e524eb8a85213a8cda5b7fff845f6,0x2d50b82f53cc24f44d562207009f89f60b04ba98bff6deba999b50b964c9dfe
200003,0x0068928198ae9c350e91761752850081262aede8cabd5824341c905b5750fae5,169911,0x01bf3b09632626f8cc8ed46a1b394df8ddf2776b5f04794ad740e335b1ec5b9a,1692881265,InterestStateUpdated,0xde0b6b3a7640000,0x0,0x63d69ae657bd2f40337c39bf35a870ac27ddf91e6623c2f52529db4c1619a51,NaN,NaN,0xde0b6b3a7640000,0x0,NaN,NaN
200004,0x02ce054477550a921fcd804daf2482d9503562dfb4790568aba6783184dd5656,175894,0x003e1ef7eca78dff0bf0e181658360ecf2a5f0d0380b5b9c1f239e3f2b057809,1693155334,Mint,NaN,NaN,NaN,interest rate updated,25007406626554445,NaN,NaN,0x057146f6409deb4c9fa12866915dd952aa07c1eb2752e451d7f3b042086bdeb8,0x1b706b18846667c44fdf2dc302ed90b98c0ae3aa6e041fdbdea1289d0284b03
